In [1]:
from base_classification import *

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
# for sampling
import random # use random.choice? and random.sample for interactions

from sklearn.metrics import make_scorer
from sklearn.datasets import make_classification
from sklearn.metrics import *


In [3]:
X, y = make_classification(2000)

In [4]:
# start...

In [5]:
current_model = BMARS(X, 2)
eval_pipeline(current_model.construct_pipeline(False), X, y)

Result: 0.7665


0.76649999999999996

In [6]:
alpha, model1 = mh_iter(X, y, current_model)

action: birth
action: birth
output: {'sign': [1, -1], 'basis': [9, 16], 'knot': array([-0.70784789,  0.19708867])}
acceptance_proba: 1.0
	bayes_factor: 1.0018326206475263
	prior_ratio: 6.346338677904939e-09
	proposal_ratio: 472713504.944928


In [7]:
alpha

True

In [8]:
model1

{'X': array([[ 0.29587129, -0.66376295,  2.28400098,  1.74521736, -0.17864501,
         -0.13268503, -1.19986506,  0.50111783, -0.07884688,  0.55941311,
         -1.17960553,  0.26435775,  2.21547411, -0.14707045,  0.22116258,
         -0.61269316,  0.18726332, -1.29042603,  0.16132383, -0.8468612 ]]),
 'basis': [[9, 16]],
 'interaction': 2,
 'params': [{'knots': array([-0.70784789,  0.19708867]), 'signs': [1, -1]}]}

In [9]:
# run for 10 iters...
iters = 50

for _ in range(iters):
    try:
        alpha, next_model = mh_iter(X, y, current_model, False)
        current_model = BMARS(**next_model)   
    except:
        print("\n---\n")
        pass
    

action: birth
action: change
action: birth
action: birth
action: change
action: death
action: change
action: change
action: birth
action: change
action: change
action: change
action: change
action: birth
action: change
action: death
action: birth
action: birth
action: death
action: birth
action: birth
action: change
action: change
action: death
action: birth
action: death
action: change
action: change
action: change
action: change
action: change
action: change
action: death
action: death
action: birth
action: birth
action: change
action: birth
action: birth
action: birth
action: change
action: birth
action: birth
action: birth
action: death
action: birth
action: death
action: change
action: change
action: change


In [10]:
eval_pipeline(current_model.construct_pipeline(False), X, y)

Result: 0.7460000000000001


0.74600000000000011

In [11]:
current_model.export()

{'X': array([[ 0.29587129, -0.66376295,  2.28400098,  1.74521736, -0.17864501,
         -0.13268503, -1.19986506,  0.50111783, -0.07884688,  0.55941311,
         -1.17960553,  0.26435775,  2.21547411, -0.14707045,  0.22116258,
         -0.61269316,  0.18726332, -1.29042603,  0.16132383, -0.8468612 ]]),
 'basis': [[12, 6],
  [1, 6],
  [12, 18],
  [1, 3],
  [7, 12],
  [3, 17],
  [0, 17],
  [0, 19],
  [2, 10],
  [1, 2]],
 'interaction': 2,
 'params': [{'knots': array([ 0.86029676,  0.14885143]), 'signs': [-1, 1]},
  {'knots': array([ 1.03064424, -0.0317254 ]), 'signs': [1, 1]},
  {'knots': array([ 0.90003421, -1.55763741]), 'signs': [1, 1]},
  {'knots': array([-0.42783983,  1.73842192]), 'signs': [1, 1]},
  {'knots': array([ 0.96155095,  1.69100481]), 'signs': [-1, 1]},
  {'knots': array([ 2.04700286,  0.19208626]), 'signs': [1, -1]},
  {'knots': array([ 0.94500333,  1.15673523]), 'signs': [-1, 1]},
  {'knots': array([ 0.19894011, -0.4824895 ]), 'signs': [-1, -1]},
  {'knots': array([-1.0